In [1]:
import os
from os import listdir 
from os.path import isfile, join
import cv2

import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Searching training data

label_folder = []
total_size = 0
data_path = r"C:\Users\ESA LAB\Desktop\Files\course\110-2\AI\03_final\resize_datasets"

#os.walk() generates the file names(dirpath, dirnames, filenames) 
#in a directory tree by walking the tree either top-down or bottom-up.
for root, dirts, files in os.walk(data_path): 
    for dirt in dirts:
        label_folder.append(dirt)
    
    total_size += len(files)

    
print("found",total_size,"files.")
print("folder:",label_folder)

found 80270 files.
folder: ['banana', 'bareland', 'carrot', 'corn', 'dragonfruit', 'garlic', 'guava', 'peanut', 'pineapple', 'pumpkin', 'rice', 'soybean', 'sugarcane', 'tomato']


In [3]:
#Load training images

base_x_train = []
base_y_train = []

for i in range(len(label_folder)):
    labelPath = data_path+r'\\'+label_folder[i]
    
    #listdir() returns a list containing the names of the entries in the directory given by path.
    #isfile() is used to check whether the specified path is an existing regular file or not.
    FileName = [f for f in listdir(labelPath) if isfile(join(labelPath, f))]
    
    for j in range(len(FileName)):
        path = labelPath+r'\\'+FileName[j]
        
        #use cv2.imread read image.
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        # img = img.astype('float')/255.0
        base_x_train.append(img)
        base_y_train.append(i)


print(np.array(base_x_train).shape)
print(np.array(base_y_train).shape)

(80270, 256, 256, 3)
(80270,)


In [4]:
base_x_train = np.array(base_x_train)
base_x_train = base_x_train.astype('float16')/255.0

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    base_x_train, base_y_train, test_size=0.2, random_state=42)

del base_x_train, base_y_train

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), padding="same", activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(128, (3, 3), padding="same", activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), padding="same", activation='relu'),
    tf.keras.layers.BatchNormalization(),    
    tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, (3, 3), padding="same", activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512, (3, 3), padding="same", activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(4,4),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(14, activation="softmax")
])

In [8]:
X_train_new, X_val, y_train_new, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

del X_train, y_train

In [9]:
from tensorflow.keras.utils import to_categorical

In [10]:
y_train_new = to_categorical(y_train_new)
y_val = to_categorical(y_val)

In [11]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)
history = model.fit(
    X_train_new,
    y_train_new,
    epochs=150,
    batch_size=20,
    validation_data=(X_val, y_val)
)


Epoch 1/150
2569/2569 [==============================] - 502s 189ms/step - loss: 5.8963 - accuracy: 0.4282 - val_loss: 2.2879 - val_accuracy: 0.4844
Epoch 2/150
2569/2569 [==============================] - 450s 175ms/step - loss: 1.8693 - accuracy: 0.6769 - val_loss: 2.8347 - val_accuracy: 0.4380
Epoch 3/150
2569/2569 [==============================] - 441s 172ms/step - loss: 0.8569 - accuracy: 0.7690 - val_loss: 0.7907 - val_accuracy: 0.7574
Epoch 4/150
2569/2569 [==============================] - 441s 172ms/step - loss: 0.5425 - accuracy: 0.8266 - val_loss: 0.8365 - val_accuracy: 0.7489
Epoch 5/150
2569/2569 [==============================] - 441s 172ms/step - loss: 0.4331 - accuracy: 0.8593 - val_loss: 0.8392 - val_accuracy: 0.7553
Epoch 6/150
2569/2569 [==============================] - 441s 172ms/step - loss: 0.3466 - accuracy: 0.8870 - val_loss: 0.7614 - val_accuracy: 0.7651
Epoch 7/150
2569/2569 [==============================] - 441s 172ms/step - loss: 0.2969 - accuracy: 0.9019

In [12]:
model.save("my_model.hdf5")
model.save_weights("model.weight")

In [13]:
prediction = model.predict(X_test)
prediction = np.argmax(prediction, axis = 1)

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prediction)*100

94.44375233586645

In [15]:
from keras.models import load_model
from sklearn.metrics import accuracy_score

In [16]:
model1 = load_model('my_model.hdf5')

In [17]:
prediction1 = model1.predict(X_test)
prediction1 = np.argmax(prediction1, axis = 1)

In [18]:
accuracy_score(y_test, prediction1)*100

94.44375233586645